In [2]:
import gc
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import optimizers, regularizers, losses

from keras.models import Model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras import metrics

import scipy
import datetime 

import keras

import sys
import pickle


import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

K.tensorflow_backend._get_available_gpus()

from configs import * 
from models import * 

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

sys.path.append("/home/nbbwang/bigdrive/AD_Project/")

/home/nbbwang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
MTL_phenotype_output_mapping = {"BRAAK":0, "CERAD":1, "PLAQUES":2, "TANGLES":3, "ABETA_IHC":4, "TAU_IHC":5}

with h5py.File(path_to_MDAD_data_folders + "%s.h5"%(full_pca_dataset), 'r') as hf:
    X = hf["ge_transformed"][:,:num_components].astype(np.float64)      
    Y = hf["labels"][:]    
    PCA_components = hf["PCA_components_"][:]
    gene_symbols = hf["gene_symbols"][:]
    labels_names = hf["labels_names"][:]

In [33]:
labels = pd.DataFrame(Y.astype(str), columns=labels_names.astype(str), dtype=str)

In [4]:
MTL_final_final_model = pickle.load(open(path_to_final_models_chosen + "MTL/final.p", "rb" ) )
method = "MTL"

In [5]:
path_to_preds = "../../Pipeline_Outputs_Submitted/final_model/MDAD_predictions/"

## Get predictions for each run

In [16]:
method = "MTL"
for i in range(100):
# for i in range(5): [for testing]
    print(i)
    fname = MTL_final_final_model
    path_to_model = final_models_save_path + "models/MTL/ACT_MSBBRNA_ROSMAP_PCA/%s/%i/200.hdf5"%(MTL_final_final_model,i)

    model = keras.models.load_model(path_to_model, custom_objects={"ordloss_cur_params": ordloss(0), \
        "ignorenans_mse": ignorenans_mse, "cat_acc": ignorenans_categorical_accuracy(0), \
        "ignorenans_scaled_mse": ignorenans_scaled_mse})


    for j,layer in enumerate(model.layers[-len(phenotypes):]):
        MTL_phenotype_output_mapping[layer.name[:-4]] = j

    preds = model.predict(X)



    if not os.path.isdir(path_to_preds):
        os.makedirs(path_to_preds)

    with h5py.File("%s%i.hdf5"%(path_to_preds,i), 'w') as hf:
        for phenotype in ["CERAD", "BRAAK", "PLAQUES", "TANGLES", "ABETA_IHC", "TAU_IHC"]:
            hf.create_dataset(phenotype, data=preds[MTL_phenotype_output_mapping[phenotype]])
    
    print("%s%i.hdf5"%(path_to_preds,i))
    K.clear_session()
    gc.collect()

0


/home/nbbwang/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


../../Pipeline_Outputs_Submitted/final_model/MDAD_predictions/0.hdf5
1
../../Pipeline_Outputs_Submitted/final_model/MDAD_predictions/1.hdf5
2
../../Pipeline_Outputs_Submitted/final_model/MDAD_predictions/2.hdf5
3
../../Pipeline_Outputs_Submitted/final_model/MDAD_predictions/3.hdf5
4
../../Pipeline_Outputs_Submitted/final_model/MDAD_predictions/4.hdf5


### Get consensus prediction (average over runs)

In [39]:
phenotypes = ['ABETA_IHC', 'BRAAK', 'CERAD', 'PLAQUES', 'TANGLES', 'TAU_IHC']
preds_runs = []
for f in os.listdir(path_to_preds):
    if f.split(".")[0].isnumeric():
        print(f)
        preds =[]
        for phenotype in phenotypes:
            with h5py.File(path_to_preds + "%i.hdf5"%i, 'r') as hf:
                preds.append(hf[phenotype][:])
        preds_runs.append(np.array(preds))
        
        
to_save_df = pd.DataFrame(np.mean(np.mean(np.array(preds_runs),axis=3),axis=0).T, columns=phenotypes)
to_save_df["sample_name"]  = labels["sample_name"]
to_save_df.to_csv(path_to_preds + "CONSENSUS_PREDICTIONS.csv")

0.hdf5
3.hdf5
1.hdf5
4.hdf5
2.hdf5
